In [1]:
# - *- coding: utf- 8*-
from LambdaNodeEmbedding import KerasEval as KE
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.callbacks import History

concate_multiplier = 2
in_dim = 5 * concate_multiplier
out_dim = 10

#These are the edges between the nodes.
current = np.array( [
                    [0,0.3,0.3,1,0.3],
                    [0.1,0.3,1,0,0.2],
                    [0,0.7,1,1,0.4],
                    [0.8,0.2,1,1,0.2]
                    ], dtype='float32')

current_ten = K.variable (value=current, dtype='float32')

################################

neigbours = np.array([
                     [0,1,1,1],
                     [0,0,0,1],
                     [0,0,0,0],
                     [0,0,0,0]
                     ], dtype='float32')

neighbouring = K.variable (value=neigbours, dtype='float32')

zero_vec = K.zeros(current[0,:].shape, dtype='float32')


################################
'''
weights = np.array([
                   [0, 0.3, 0.1, 1, 0.3],
                   [1, 0.1, 0.1, 1, 0.9], 
                   [0, 0.3, 0.1, 0.9, 0.3],
                   [0.6, 0.3, 1, 0.7, 0.1]
                   ], dtype='float32')
'''

weight_array = np.random.rand(in_dim,out_dim)
weights_ten = K.variable (value=weight_array, dtype='float32')

print('##### Load Variables #####')

##### Load Variables #####


Using TensorFlow backend.


In [2]:
print('##### Load Support Functions #####')

def ConsoleReporter(label, printable,evaluate=True):
    if evaluate and AssertIsTensor(printable):
        print(label,':\n',KE(printable),'\n')
    else:
        print(label,':\n',printable,'\n')
        
def AssertTensorShapeEqual(tensor_x, tensor_y):
    assert (tensor_x.shape == tensor_y.shape), ('Assertion failed shapes dont match! [',tensor_x.shape,'] dont match " [',tensor_y.shape,']')
    
def AssertVectorLength(vector_x, vector_y):
    assert (vector_x.shape[0] == vector_y.shape[0]), ('Vector lenghts dont match! [',vector_x.shape[0],'] dont match " [',vector_y.shape[0],']')    
    
def AssertTensorDotDim(tensor_x, tensor_y):
    x_dim_0 = tensor_x.shape[0]
    y_dim_1 = tensor_y.shape[1]
    assert (x_dim_0 == y_dim_1),('Assertion failed for matrix multiplication cause [',x_dim_0,'] * [',y_dim_1,'] dont match!')
    
def AssertIsTensor(tensor):
    return K.is_tensor(tensor)

def AssertIsKerasTensor(tensor):
    return K.is_keras_tensor(tensor)

##### Load Support Functions #####


In [3]:
print('##### Load Aggregation Step Functions #####')
def MaxPoolAggregator(vectors, axis=0):
    return K.max(vectors, axis)

def MeanAggregator(vectors, axis=0):
    return K.mean(vectors, axis)

def PerformAggregator(found_vectors, features_shape, axis=0, aggregator='mean'):
    aggregated = None
    
    if (aggregator=='mean'):
        aggregated = MeanAggregator(found_vectors, axis)
    else:
        aggregated = MaxPoolAggregator(found_vectors, axis)
            
    AssertTensorShapeEqual(aggregated, features_shape)
    return aggregated


def GetSingleVectorFeatures(max_neighbours, vertex_neighbours, last_features):
    found_neigbours = []
    
    if KE(K.sum(vertex_neighbours)) > 0:
        for k in range(max_neighbours):
                if vertex_neighbours[k].eval() > 0.999999:
                    found_neigbours.append(last_features[k, :])

        assert (len(found_neigbours) == KE(K.sum(vertex_neighbours))), ('Amount of existing neigbours did not match the found neigbours count!') 
        
    return found_neigbours


def GetAllVectorsFeatures(iterator, last_features, in_neighbouring, axis=0, aggregator='mean'):
    updated_features = []
    for i in range(iterator):
        vertex_neighbours = in_neighbouring[i,:]
        max_neighbours = vertex_neighbours.shape[0]
        found_neigbours = GetSingleVectorFeatures(max_neighbours, vertex_neighbours, last_features)
        
        if len(found_neigbours) < 1: found_neigbours.append(zero_vec)
             
        found_vectors = K.variable(value=found_neigbours, dtype='float32')
        updated_features.append(PerformAggregator(found_vectors, last_features[0,:], 0, aggregator))
        
        
    final_features = K.stack(updated_features)
    AssertTensorShapeEqual(last_features, final_features)
    return final_features


def GetNewFeatures(last_features, in_neighbouring):
    AssertIsTensor(last_features)
    AssertIsTensor(in_neighbouring)
    AssertTensorDotDim(last_features, in_neighbouring)
    final_features = GetAllVectorsFeatures(in_neighbouring.shape[0],
                                           last_features, 
                                           in_neighbouring,
                                           0,
                                           'max')
    return final_features
        

##### Load Aggregation Step Functions #####


In [4]:
print('##### Load Concatenation Step Functions #####')

def ControledConcatenation(prev_features, new_features, axis=1):
    AssertIsTensor(prev_features)
    AssertIsTensor(new_features)
    AssertVectorLength(prev_features[0,:], new_features[0,:])
    return K.concatenate([prev_features, new_features], axis)    

##### Load Concatenation Step Functions #####


In [5]:
print('##### Execute #####')


sess = tf.Session()
K.set_session(sess)
init = tf.global_variables_initializer()
with sess:
    init.run()
    init_vecs = current_ten
    #concat_vecs = K.concatenate([init_vecs, next_vecs], axis=1)
    
    #ConsoleReporter('Old_Features', init_vecs, True)
    #ConsoleReporter('Next_Features', next_vecs, True)
    #ConsoleReporter('N_Concatenation',concat_vecs,True)

    ConsoleReporter('Inits',init_vecs,True)
    ConsoleReporter('Neighbouring',neighbouring,True)
    agg_features = GetNewFeatures(init_vecs, neighbouring)
    ConsoleReporter('Results',agg_features,False)
    
    concatenated = ControledConcatenation(init_vecs, agg_features)
    ConsoleReporter('concatenated',concatenated,False)
    
    print(KE(weights_ten))
    

##### Execute #####
Inits :
 [[0.  0.3 0.3 1.  0.3]
 [0.1 0.3 1.  0.  0.2]
 [0.  0.7 1.  1.  0.4]
 [0.8 0.2 1.  1.  0.2]] 

Neighbouring :
 [[0. 1. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

Results :
 Tensor("stack:0", shape=(4, 5), dtype=float32) 

concatenated :
 Tensor("concat:0", shape=(4, 10), dtype=float32) 

[[0.2975325  0.8380124  0.7189067  0.5091476  0.0704146  0.83534306
  0.64218426 0.67020065 0.3281497  0.37891614]
 [0.87482077 0.73889554 0.7336551  0.9914017  0.3494411  0.40586993
  0.13665293 0.55139637 0.47434765 0.07291498]
 [0.9437597  0.35771915 0.29451755 0.5256096  0.43880516 0.01528344
  0.740657   0.48041975 0.90161043 0.36623594]
 [0.8996198  0.12548134 0.47848797 0.33944732 0.10810245 0.8927751
  0.40413842 0.00495444 0.8827185  0.4213006 ]
 [0.24523854 0.7156841  0.8110689  0.3720916  0.39917877 0.39250183
  0.48664686 0.03899949 0.16208604 0.78932136]
 [0.13245223 0.22343852 0.37164304 0.12205135 0.7486904  0.01197008
  0.31509995 0.5491979  0.48